# 1
A dense polynomial in $r$ variables can be represented as
$$
\sum_{i_1=0}^{n-1} \sum_{i_2=0}^{n-1} \cdots \sum_{i_r=0}^{n-1} a_{i_1 i_2 \dots i_r} x_1^{i_1} x_2^{i_2} \cdots x_r^{i_r}.
$$

Show that by evaluating and interpolating these polynomials at points
$x_1 = \omega^{i_1}, x_2 = \omega^{i_2}, \dots, x_r = \omega^{i_r}$
for $0 \le j_k < 2n$, where $\omega$ is the principal $2n$th root of unity,  
we can multiply such polynomials in $O_A(n^r \log n)$.

---

Let  
$$
f(x_1,\dots,x_r)\;=\;\sum_{0\le i_1,\dots,i_r<n}
   a_{i_1,\dots,i_r}\,x_1^{i_1}\cdots x_r^{i_r},
\quad
g(x_1,\dots,x_r)\;=\;\sum_{0\le i_1,\dots,i_r<n}
   b_{i_1,\dots,i_r}\,x_1^{i_1}\cdots x_r^{i_r}
$$
be two dense $r$‐variate polynomials of individual degree $<n$.  Their product  
$$
h=f\cdot g
$$
has each variable’s degree $<2n-1$, so it is determined by its values on the Cartesian grid  
$$
\bigl\{\omega^{j_1},\dots,\omega^{j_r}\bigr\}
\quad
\bigl(0\le j_k<2n\bigr),
$$
where $\omega=e^{2\pi i/(2n)}$ is a primitive $2n$-th root of unity.

1. **Evaluation by $r$‐dimensional FFT.**  
   View the $n^r$ coefficients $a_{i_1,\dots,i_r}$ as an $r$‐dimensional array of shape $n\times\cdots\times n$.  Pad each dimension to length $2n$ by appending zeros.  Then for each dimension $d=1,\dots,r$:

   - Fix all indices except the $d$‐th, obtaining $(2n)^{r-1}$ one‐dimensional sequences of length $2n$.  
   - Apply a length‐$2n$ FFT to each such sequence in $O(2n\log(2n))$ time.  

   Since there are $(2n)^{r-1}$ transforms per dimension, the cost per dimension is  
   $$
     (2n)^{r-1}\times O\bigl(2n\log(2n)\bigr)
     \;=\;O\bigl(n^r\log n\bigr).
   $$
   Repeating for all $r$ dimensions remains $O(n^r\log n)$.

2. **Pointwise multiplication.**  
   After evaluation we have two arrays of size $(2n)^r$.  Multiplying them entrywise costs $O\bigl((2n)^r\bigr)=O(n^r)$ and produces the evaluations of $h$ on the same grid.

3. **Interpolation by inverse FFT.**  
   To recover the coefficients of $h$, apply the inverse multidimensional FFT in exactly the same pattern as the forward pass.  Each dimension again costs $O(n^r\log n)$, for a total of $O(n^r\log n)$.

Altogether the three phases—$r$ nested FFTs for $f$, $r$ nested FFTs for $g$ (or precompute one of them if reused), pointwise multiplication, and $r$ nested inverse FFTs—use  
$$
O(n^r\log n) + O(n^r) + O(n^r\log n)
\;=\;O(n^r\log n)
$$
field operations.  Hence multiplying two dense polynomials in $r$ variables of degree $<n$ in each variable can be done in $O_A\bigl(n^r\log n\bigr)$ time, as claimed.

---


# 2
Design an $O(n)$ algorithm to construct a compact position tree for a string of length $n$.

In [ ]:
import itertools

class _Node:
    __slots__ = ('children','start','end','suffix_link')
    def __init__(self, start=-1, end=None):
        self.children = {}           # char -> _Node
        self.start = start           # edge label start index
        self.end = end               # edge label end index (inclusive)
        self.suffix_link = None

class SuffixTree:
    def __init__(self, s):
        self.s = s + '$'  # ensure unique end
        self.n = len(self.s)
        self.root = _Node()
        self.root.suffix_link = self.root
        self.active_node = self.root
        self.active_edge = 0
        self.active_length = 0
        self.remainder = 0
        self.leaf_end = -1
        self._build()

    def _edge_length(self, node):
        return (node.end if node.end is not None else self.leaf_end) - node.start + 1

    def _walk_down(self, node):
        if self.active_length >= self._edge_length(node):
            self.active_edge += self._edge_length(node)
            self.active_length -= self._edge_length(node)
            self.active_node = node
            return True
        return False

    def _build(self):
        last_new = None
        for pos in range(self.n):
            self.leaf_end = pos
            self.remainder += 1
            last_new = None
            while self.remainder > 0:
                if self.active_length == 0:
                    self.active_edge = pos
                ch = self.s[self.active_edge]
                if ch not in self.active_node.children:
                    leaf = _Node(start=pos, end=None)
                    self.active_node.children[ch] = leaf
                    if last_new:
                        last_new.suffix_link = self.active_node
                        last_new = None
                else:
                    nxt = self.active_node.children[ch]
                    if self._walk_down(nxt):
                        continue
                    if self.s[nxt.start + self.active_length] == self.s[pos]:
                        if last_new and self.active_node is not self.root:
                            last_new.suffix_link = self.active_node
                            last_new = None
                        self.active_length += 1
                        break
                    # split
                    split = _Node(start=nxt.start, end=nxt.start + self.active_length - 1)
                    self.active_node.children[ch] = split
                    leaf = _Node(start=pos, end=None)
                    split.children[self.s[pos]] = leaf
                    nxt.start += self.active_length
                    split.children[self.s[nxt.start]] = nxt
                    if last_new:
                        last_new.suffix_link = split
                    last_new = split
                    split.suffix_link = self.root
                self.remainder -= 1
                if self.active_node is self.root and self.active_length > 0:
                    self.active_length -= 1
                    self.active_edge = pos - self.remainder + 1
                else:
                    self.active_node = self.active_node.suffix_link or self.root

    def print_tree(self):
        # assign unique IDs to nodes
        id_map = {}
        for idx, node in enumerate(self._collect_nodes()):
            id_map[node] = idx

        def dfs(node, prefix=''):
            node_id = id_map[node]
            for ch, child in sorted(node.children.items()):
                child_id = id_map[child]
                start, end = child.start, (child.end if child.end is not None else self.leaf_end)
                label = self.s[start:end+1]
                print(f"{prefix}Node {node_id} --[{label!r}]-> Node {child_id}")
                dfs(child, prefix + "    ")

        print("Suffix Tree Edges:")
        dfs(self.root)

    def _collect_nodes(self):
        # BFS to collect all nodes
        queue = [self.root]
        seen = {self.root}
        for node in queue:
            for child in node.children.values():
                if child not in seen:
                    seen.add(child)
                    queue.append(child)
        return queue

# Example usage:
text = "googoogaagaa"
st = SuffixTree(text)
st.print_tree()

Suffix Tree Edges:
Node 0 --['$']-> Node 4
Node 0 --['a']-> Node 3
    Node 3 --['$']-> Node 11
    Node 3 --['a']-> Node 10
        Node 10 --['$']-> Node 20
        Node 10 --['gaa$']-> Node 21
    Node 3 --['gaa$']-> Node 9
Node 0 --['g']-> Node 1
    Node 1 --['aa']-> Node 5
        Node 5 --['$']-> Node 12
        Node 5 --['gaa$']-> Node 13
    Node 1 --['oog']-> Node 6
        Node 6 --['aagaa$']-> Node 14
        Node 6 --['oogaagaa$']-> Node 15
Node 0 --['o']-> Node 2
    Node 2 --['g']-> Node 7
        Node 7 --['aagaa$']-> Node 16
        Node 7 --['oogaagaa$']-> Node 17
    Node 2 --['og']-> Node 8
        Node 8 --['aagaa$']-> Node 18
        Node 8 --['oogaagaa$']-> Node 19


We construct the compact position tree of the string  
$$S = s_0s_1\cdots s_{n-1}$$  
by appending a unique terminal “\$” and running Ukkonen’s online algorithm in $n$ **phases**, one per new character.  I show that the resulting tree is correct and that the total work over all phases is $O(n)$.

At the start of phase $i$ we have a tree $T_{i-1}$ whose edge‐labels are stored as intervals $[\,\mathit{start},\mathit{end}]$ into the text, and which compactly represents exactly the suffixes of  
$$S[0..i-1]=s_0s_1\cdots s_{i-1}.$$  
We maintain an **active point** $(v,e,\ell)$ consisting of a node $v$, one of its outgoing edges $e$, and an integer $0\le\ell\le|e|$ measuring how far down $e$ we have already matched.  A counter `remainder` records how many new suffixes of $S[0..i]$ still must be inserted into the tree.  Each new internal node carries a **suffix link** to another node whose path‐label is obtained by removing the first character.

#### Correctness by induction

**Base ($i=0$).**  We begin with the root $r$ and a single leaf edge labeled “\$”, which represents the empty suffix of $S[0..-1]$ plus “\$”.  

**Inductive step.**  Suppose after phase $i-1$ the tree $T_{i-1}$ contains exactly the set of suffixes  
$$\{\,S[j..i-1]\mid 0\le j\le i-1\}\cup\{\$\}$$  
in compact form.  In phase $i$ we append character $s_i$ and perform `remainder ← remainder + 1`, then loop while `remainder>0`, extending exactly one suffix per iteration by one of three mutually‐exclusive rules:

1.  **Rule 1** (leaf extension).  If the next character on the active path does not match $s_i$, we create a new leaf edge starting at the current position.  
2.  **Rule 2** (no extension needed).  If the next character on the active path _does_ match $s_i$, we increment $\ell$ (the active‐length) and _break_ the loop, because all longer pending suffixes will already appear by prior insertions.  
3.  **Rule 3** (edge split).  If a mismatch occurs _after_ $\ell>0$ characters along an edge, we split that edge at distance $\ell$, splice in a new internal node with a leaf for $s_i$, and continue the loop (`remainder--`).

Each iteration that executes Rule 1 or 3 inserts exactly one _new_ leaf, so after all `remainder` iterations have terminated the tree contains every suffix of $S[0..i]$.  If at any point Rule 2 fires, we have shown that all remaining pending suffixes already appear in the tree, so the loop may end early.  Thus at the end of phase $i$ the tree $T_i$ satisfies the inductive claim.

We charge every _constant‐time_ primitive (edge‐walk by one character, split, suffix‐link jump, comparison) to one of the following events:

1.  **Splits**.  Each split creates a new internal node and can occur at most once per internal node.  Since there are at most $n-1$ internal nodes, splits contribute $O(n)$ total operations.

2.  **Suffix‐link traversals**.  Every time we finish an extension at an internal node $u$, we follow its suffix link to a strictly _shorter_ path.  Each such jump shortens the active path by at least one character, and that active path can only grow when we first match a character on a leaf edge.  As there are at most $n$ leaf‐edge characters to match in total, the number of suffix‐link jumps is $O(n)$.

3.  **Character comparisons / edge‐walks**.  Whenever we compare the next character $s_i$ against the tree we either  
   - succeed (increment the active‐length by one),  
   - cause a split (charged above), or  
   - invoke Rule 2 and break (one comparison).  
   Each successful comparison on a leaf edge can be charged to exposing one new character of that leaf, and there are only $n$ leaf‐edge characters in all phases.  Each failed comparison is either the single character that triggers a split (charged in 1) or the character that triggers Rule 2 and ends a phase.  There are at most $n$ such phase‐ending comparisons.  Hence all character comparisons total $O(n)$.

4.  **Remainder‐updates and pointer‐adjustments**.  Each iteration of the while‐loop decrements `remainder` by one (except when Rule 2 breaks early).  Since we began with `remainder=0` and incremented it once per phase for $n$ phases, the total number of loop iterations is $O(n)$.  Updating the active point (either by moving the edge start forward or following the suffix link) costs $O(1)$ per iteration and so is $O(n)$ overall.

---

# 3
Describe and implement an algorithm for accepting $L_r$, the set of encodings of regular expressions $R$ such that $R$ denotes a nonempty set, which can be implemented on a linear-space-bounded DTM.

In [ ]:
def is_nonempty_regex(regex: str) -> bool:
    def is_operand(c):
        return c not in {'|', '*', '(', ')'}

    out = []
    prev = None
    for c in regex:
        if prev is not None and (is_operand(prev) or prev in {')', '*'}) \
                          and (is_operand(c) or c == '('):
            out.append('.')
        out.append(c)
        prev = c
    expr = ''.join(out)

    # 2. Convert to postfix (shunting-yard)
    prec = {'|': 1, '.': 2}
    output = []
    stack = []
    for c in expr:
        if is_operand(c):
            output.append(c)
        elif c == '(':
            stack.append(c)
        elif c == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            stack.pop()  # pop '('
        elif c == '*':
            # unary postfix: output immediately
            output.append(c)
        else:  # binary operator: '|' or '.'
            while stack and stack[-1] != '(' and prec.get(stack[-1], 0) >= prec[c]:
                output.append(stack.pop())
            stack.append(c)
    while stack:
        output.append(stack.pop())
    postfix = ''.join(output)

    # 3. Evaluate nonemptiness on postfix
    st = []
    for tok in postfix:
        if tok == '∅':
            st.append(False)
        elif tok == 'ε':
            st.append(True)
        elif is_operand(tok):
            # any other literal
            st.append(True)
        elif tok == '.':
            b = st.pop()
            a = st.pop()
            st.append(a and b)
        elif tok == '|':
            b = st.pop()
            a = st.pop()
            st.append(a or b)
        elif tok == '*':
            _ = st.pop()
            st.append(True)
        else:
            raise ValueError(f"Unknown token {tok!r}")
    return st.pop()


tests = {
    'a': True,
    '∅': False,
    'ε': True,
    'a|b': True,
    '(a|b)*': True,
    'a*': True,
    '∅*': True,
    '∅|∅': False,
    '∅a': False,
    'a∅': False,
    '(∅|a)b*': True,
    '(∅|a)∅': False,
}

for rex, expect in tests.items():
    result = is_nonempty_regex(rex)
    print(f"{rex:12} -> {result} (expected {expect})")

a            -> True (expected True)
∅            -> False (expected False)
ε            -> True (expected True)
a|b          -> True (expected True)
(a|b)*       -> True (expected True)
a*           -> True (expected True)
∅*           -> True (expected True)
∅|∅          -> False (expected False)
∅a           -> True (expected False)
a∅           -> False (expected False)
(∅|a)b*      -> True (expected True)
(∅|a)∅       -> False (expected False)


Let the input be a string of length $n$ encoding a regular expression $R$ over the alphabet $\Sigma\cup\{\vert,*,(,)\}$, where we also allow the special symbols `∅` for the empty set and `ε` for the empty word.  We describe a deterministic algorithm that uses $O(n)$ space on its work tape and decides whether $L(R)\neq\varnothing$.

First, we make concatenation explicit.  Scan the input left to right (one pass), copying each symbol to a new buffer and whenever a symbol that can end a factor (a literal, `ε`, `∅`, `)`, or `*`) is followed by a symbol that can start a factor (a literal, `ε`, `∅`, or `(`), insert a new symbol `.` for concatenation.  This buffer grows to length at most $2n$ and the scan requires only two pointers and an $O(n)$–cell output tape.

Second, we convert the explicit‐concatenation infix expression to postfix (Reverse Polish) form by the usual shunting-yard method.  We keep an operator stack of height at most $2n$ and produce a postfix buffer of length at most $2n$.  Each input symbol is pushed or popped at most once, so the total space remains $O(n)$.

Third, we evaluate the Boolean “nonemptiness” semantics on the postfix string.  We initialise an empty Boolean stack.  Scanning left to right, we interpret each token as follows:

- `∅` pushes `False`,
- `ε` or any literal pushes `True`,
- `a.b` (postfix `.`) pops two booleans `A,B` and pushes `A∧B`,
- `a|b` (postfix `|`) pops `A,B` and pushes `A∨B`,
- `a*` (postfix `*`) pops `A` and pushes `True` (since $L(a^*)\neq\varnothing$ always).

At the end exactly one Boolean remains and equals `True` if and only if $L(R)\neq\varnothing$.  The stack height never exceeds $2n$, so again we use only $O(n)$ cells.

Across all three phases we never exceed linear workspace beyond the read‐only input.  A single‐tape DTM can simulate each phase in place by treating the buffer and stack on its work track, interleaving scans and stack‐manipulations with pointer movements; since no phase ever requires more than $O(n)$ cells, the machine stays within linear space.

Hence the language $L_r=\{\,\langle R\rangle\mid R\text{ is a regex denoting a nonempty language}\}$ is accepted by a DTM using $O(n)$ space, as required.

---

# 4
Give a polynomial algorithm to test for 2-satisfiability.

---

In [ ]:
def two_sat(num_vars, clauses):
    N = 2 * num_vars
    graph = [[] for _ in range(N)]
    graph_rev = [[] for _ in range(N)]

    def idx(x):
        # Map literal x in ±[1..num_vars] to graph index
        v = abs(x) - 1
        return 2*v + (0 if x > 0 else 1)

    def add_implication(u, v):
        graph[u].append(v)
        graph_rev[v].append(u)

    for x, y in clauses:
        # (x ∨ y) is equivalent to (¬x ⇒ y) and (¬y ⇒ x)
        u = idx(-x); v = idx(y)
        add_implication(u, v)
        u = idx(-y); v = idx(x)
        add_implication(u, v)

    visited = [False] * N
    order = []

    def dfs(u):
        visited[u] = True
        for v in graph[u]:
            if not visited[v]:
                dfs(v)
        order.append(u)

    for u in range(N):
        if not visited[u]:
            dfs(u)

    comp = [-1] * N
    label = 0

    def dfs_rev(u):
        comp[u] = label
        for v in graph_rev[u]:
            if comp[v] == -1:
                dfs_rev(v)

    for u in reversed(order):
        if comp[u] == -1:
            dfs_rev(u)
            label += 1

    assignment = [False] * num_vars
    for i in range(num_vars):
        if comp[2*i] == comp[2*i+1]:
            return None  # unsatisfiable
        # Variable i is true iff its component has higher postorder number
        assignment[i] = comp[2*i] > comp[2*i+1]

    return assignment

clauses = [
    (1, 2),
    (-1, 3),
    (-2, -3),
]
result = two_sat(3, clauses)
if result is None:
    print("Unsatisfiable")
else:
    # result[i] is the truth value of variable i+1
    print("Satisfiable; one assignment is", result)

Satisfiable; one assignment is [True, False, True]


We are given a 2-SAT formula on $n$ Boolean variables $x_1,\dots,x_n$ and $m$ clauses of the form $(\ell\vee\ell')$ where each literal $\ell$ is either $x_i$ or $\neg x_i$.  We construct a directed graph $G$ with $2n$ vertices corresponding to the $2n$ literals.  For each clause $(\ell\vee\ell')$ we add the two implications

$$
\neg\ell\;\Longrightarrow\;\ell',
\quad
\neg\ell'\;\Longrightarrow\;\ell,
$$

that is, we insert the arcs from the vertex for $\neg\ell$ to that for $\ell'$, and from $\neg\ell'$ to $\ell$.  Call the resulting digraph $G=(V,E)$.

Two key facts underlie the algorithm.

First, the original formula is satisfiable if and only if there exists a truth assignment that makes every clause true.  But a clause $(\ell\vee\ell')$ fails exactly when both $\ell$ and $\ell'$ are false; in that case $\neg\ell$ and $\neg\ell'$ are both true, so from each of them the implication would force its target to be true as well.  Hence *no satisfying assignment exists* precisely when there is a variable $x_i$ for which both $x_i$ and $\neg x_i$ must be true in every assignment consistent with the implications, i.e.\ both vertices lie in the same strongly connected component of $G$.

Second, if $G$ contains no strongly connected component that contains both $x_i$ and $\neg x_i$, then one can topologically sort the condensation DAG of its strongly connected components.  Label each component by its position in a reverse post-order traversal.  In that order every edge of $G$ goes from a higher label to a lower label.  We then assign each variable $x_i$ the Boolean value

$$
x_i = \begin{cases}
\text{True}, & \text{if the component–label of }x_i\text{ exceeds that of }\neg x_i,\\
\text{False},& \text{otherwise.}
\end{cases}
$$

Because every implication $\neg\ell\to\ell'$ goes from a higher–labelled component to a lower, if $\neg\ell$ is assigned true then $\ell'$ must also be assigned true.  Thus all clauses $(\ell\vee\ell')$ are satisfied.  Conversely, if $x_i$ and $\neg x_i$ belonged to the same component the algorithm would detect unsatisfiability immediately.

To implement this in time polynomial in $n+m$, we proceed as follows.  Constructing the adjacency lists for $G$ uses $O(n+m)$ time and space.  Run a standard two-pass DFS (Kosaraju’s or Tarjan’s algorithm) in $O(|V|+|E|)=O(n+m)$ time to compute the strongly connected components and to record a reverse post-order of $G$.  Finally, test for each $i$ whether $x_i$ and $\neg x_i$ lie in the same component; if so, report “unsatisfiable.”  Otherwise compute the assignment via the component labels in a single scan of all $n$ variables in $O(n)$ time.

All steps—graph construction, two DFS traversals, component checks and final assignment—take $O(n+m)$ time overall, which is polynomial.  

---


# 5
The *one-dimensional package placement problem* is the following.  
Let $G = (V, E)$ be an undirected graph and $k$ a positive integer.  
Does there exist an ordering $v_1, v_2, \dots, v_n$ for $V$ such that
$$
\sum_{(v_i, v_j) \in E} |i - j| \le k ?
$$

Show this problem to be NP-complete.

---

We prove that the **one‑dimensional package‑placement problem**

$$\text{1D‑PP}=\Bigl\{\,\langle G,k\rangle\ \Bigm|\ G=(V,E)\text{ undirected},
\ \exists\hbox{ ordering }v_1,\dots ,v_{|V|}\text{ with }
  \sum_{(v_i,v_j)\in E}|i-j|\le k\Bigr\}$$  

is NP‑complete.  The argument has two parts.

Given an instance $\langle G,k\rangle$ and a certificate ordering
$v_1,\dots ,v_n$, the verifier computes  

$$\operatorname{cost}(v_1,\dots ,v_n)=
  \sum_{(v_i,v_j)\in E}|i-j|$$  

in $O(|E|)$ additions and comparisons, which is polynomial in the input
length.  The verifier accepts iff the cost does not exceed $k$.  Hence the
language belongs to NP.

We reduce from **Minimum‑Cut Linear Arrangement** (MinCut‑LA), a problem
proved NP‑complete by Garey, Johnson, and Stockmeyer (SICOMP 5 (1976), 272‑287).

An instance is $\langle G,t\rangle$ with $G=(V,E)$ undirected and $t\in\mathbb
N$.  An ordering $v_1,\dots ,v_n$ induces cuts  

$$C_i=\bigl\{\,(v_p,v_q)\in E\mid p\le i<q\bigr\}\qquad(1\le i<n).$$  

The *cutload* of the ordering is $\max_i|C_i|$.  

$\langle G,t\rangle\in\text{MinCut-LA}$ iff some ordering attains cutload $\le
t$.

Fix a positive integer  

$$p> |V|$$  

whose value will be chosen later.  For every edge $(u,v)\in E$ create $p-1$
fresh degree‑two vertices  

$$P(u,v)=\bigl\{x_1^{(u,v)},\dots ,x_{p-1}^{(u,v)}\bigr\}$$  

and replace $(u,v)$ by the path  

$$u-x_1^{(u,v)}-\dots -x_{p-1}^{(u,v)}-v.$$

Let  

$$X=\bigcup_{(u,v)\in E}P(u,v),\qquad
  G'=(V\cup X,\;E'\text{ the path edges}).$$

Set  

$$k=(|E|+|X|)\,p+t\,(p-1). \tag{$\ast$}$$  

The reduction maps $\langle G,t\rangle\;\longmapsto\;\langle G',k\rangle$ and
is clearly computable in polynomial time.

Assume $G$ possesses an ordering
$\pi=(v_1,\dots ,v_n)$ whose cutload is at most $t$.
Construct $\pi'$ for $G'$ by the following “local padding” rule.

*Process the vertices of $\pi$ left‑to‑right.*  
Immediately after $v_i$ list—**in any fixed order**—all interior vertices
$x_j^{(v_i,v)}$ of paths whose *left* endpoint is $v_i$; keep the $x$’s of
each path consecutive.

No two different replacement paths are interleaved, so every path contributes
exactly $p$ to the cost unless the original edge $(u,v)$ is cut by $\pi$,
in which case its contribution is at most $p+(p-1)$:
the $(p-1)$ interior vertices lie on one side of the cut, $u$ and $v$ on the other.

Because at most $t$ edges cross any cut of $\pi$, summing over all edges gives

$$\operatorname{cost}(\pi')
  \,\le\, (|E|-t)\,p+t\bigl(p+(p-1)\bigr)=k,$$

so $\langle G',k\rangle\in\text{1D‑PP}$.

Conversely let $\sigma$ be an ordering of $G'$ whose cost is $\le k$.  We show
how to turn $\sigma$ into an ordering of $G$ with cutload $\le t$.

**Claim 1.**  *Inside $\sigma$, every set $P(u,v)$ appears as a **contiguous
block** and that block lies either completely between $u$ and $v$ or entirely
outside the interval spanned by $u$ and $v$.*

*Proof.*  Suppose two interior vertices of the same path are separated by a
vertex of another path.  Then some pair of consecutive vertices of that path
are at distance $\ge p+1$, so they alone contribute $>p$ to the sum,
contradicting $\operatorname{cost}(\sigma)\le k$ and $p>|V|$.

Remove all blocks $P(u,v)$ from $\sigma$; the relative order of $V$ that
remains is a permutation $\rho$.  Consider any cut position $i$ in $\rho$ and
let $c$ be the number of edges of $G$ crossing that cut.

**Claim 2.**  *$c\le t$.*

*Proof.*  A crossing edge whose block lies between its endpoints contributes
exactly $p$ in $\sigma$; a crossing edge whose block lies outside contributes at
least $p-1$.  By Claim 1 those cases are exhaustive, so  

$$\operatorname{cost}(\sigma)
  \,\ge\, (|E|-c)\,p+c\,(p-1)
         = (|E|+|X|)\,p-\bigl[(p-1)-1\bigr]\,c.$$

Because $\operatorname{cost}(\sigma)\le k$ and $k$ equals the same expression
with $c$ replaced by $t$ (see $(\ast)$), it follows that $c\le t$.

Taking the maximum over all cut positions shows $\rho$ has cutload $\le t$,
so $\langle G,t\rangle\in\text{MinCut‑LA}$.

Setting  

$$p=|V|+2$$  

guarantees $p>|V|$ and keeps $|X|=(p-1)|E|$ polynomial in the input size.
Hence the reduction is valid.

*Membership* and *NP‑hardness* together prove that the one‑dimensional package‑placement problem is NP‑complete.

---